In [1]:
from decode import *

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### It seems that, with AlphaMonarch model, the phenomenom is not obvious on the displayed cases.

In [2]:
query="Question: A coin is heads up. Fletcher flips the coin. Conception flips the coin. Is the coin still heads up? \nAnswer: "
k_response = get_k_path_prob(model, tokenizer, query, k=3)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/Users/fangyuanyu/Implementation/CoT-decoding/decode.py:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  k_prob = torch.nn.functional.softmax(logit[0][logit[0].argsort()[-k:]])
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/Users/fangyuanyu/Implementation/CoT-decoding/decode.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  path_prob = torch.nn.functional.softmax(path_prob)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[(',', tensor(6.1817e-15)), ('the', tensor(1.3026e-08)), ('final', tensor(1.)), ('outcome', tensor(1.2931e-07)), ('depends', tensor(1.0049e-05)), ('on', tensor(1.0518e-07)), ('the', tensor(1.2783e-07)), ('sequential', tensor(2.0955e-07)), ('flips', tensor(1.2241e-05)), ('and', tensor(1.7349e-06)), ('luck,', tensor(9.2372e-07)), ('not', tensor(0.0024)), ('fixed', tensor(6.7402e-10)), ('in', tensor(4.2862e-09)), ('the', tensor(2.0739e-09)), ('given', tensor(1.5983e-08)), ('scenario.', tensor(1.0193e-07)), ('The', tensor(9.4026e-08)), ('coin', tensor(4.5586e-07)), ('may', tensor(1.0442e-07)), ('land', tensor(3.2894e-06)), ('heads', tensor(9.8464e-09)), ('again,', tensor(7.6625e-07)), ('or', tensor(4.1860e-06)), ('it', tensor(7.6430e-08)), ('may', tensor(8.0238e-07)), ('land', tensor(1.7521e-07)), ('tails', tensor(2.0327e-06)), ('after', tensor(2.7625e-05)), ("Fletcher's", tensor(0.0004)), ('or', tensor(0.0052)), ("Conception's", tensor(0.2318)), ('flip.', tensor(0.0018)), ('Without', tens

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[('Only', tensor(6.1817e-15)), ('if', tensor(1.3026e-08)), ('luck', tensor(1.)), ('maintained', tensor(3.3580e-06)), ('the', tensor(1.7344e-08)), ('identical', tensor(1.6870e-05)), ('outcome', tensor(3.7768e-06)), ('for', tensor(2.4614e-06)), ('both', tensor(9.9315e-07)), ('the', tensor(6.7521e-07)), ('flips.', tensor(1.0328e-06)), ('In', tensor(1.7283e-06)), ('real', tensor(1.7710e-05)), ('life,', tensor(0.0006)), ("it's", tensor(5.7719e-07)), ('random', tensor(9.8471e-07)), ('for', tensor(1.6904e-06)), ('each', tensor(2.0447e-06)), ('flip,', tensor(0.1268)), ('so', tensor(8.4270e-07)), ('probability-wise,', tensor(0.0001)), ('heads', tensor(1.3745e-07)), ('may', tensor(1.4398e-07)), ('or', tensor(0.0002)), ('may', tensor(7.8593e-06)), ('not', tensor(4.1778e-06)), ('persist.', tensor(5.1917e-05)), ('', tensor(0.0068)), ('', tensor(4.7502e-06)), ('In', tensor(0.0788)), ('a', tensor(3.0858e-08)), ('hypothetical', tensor(1.9370e-07)), ('scenario', tensor(6.4870e-08)), ('where', tensor(0.